# [CAADRIA 2019] Multi Objective Evolutionary Algorithms - PF processing 

In [1]:
# https://plot.ly/python/axes/
import pandas as pd
import numpy as np

In [161]:
# Files Configurations
files_header = False
base_folder = "final/outputs/algorithms/"
output_folder = "final/outputs/"

# Get the data (csv file should be on the same folder as this file)
def get_algorithm_data(algorithm, parent_folder=base_folder):
    file = parent_folder + "all_" + algorithm + "_solutions.csv"
    print("Reading file: " + file)
    return pd.read_csv(file, sep=',', header=None)

In [17]:
# Variables and Objective columns
vars_cols = [3, 4, 5, 6, 7, 8]
objs_cols = [9, 10]

# Drop columns (Time total, t1, t2)
drop_cols = [0, 1, 2]

In [18]:
# Objectives
o1 = objs_cols[0]
o2 = objs_cols[1]

In [19]:
# Algorithms 
algorithms = ["MOEAD", "SPEA2", "NSGAII", "OMOPSO", "SMPSO", "PESA2", "PAES", "EpsMOEA", "GDE3", "CMAES"]
#algorithms = ["SMPSO"]
#algorithms = ["MOEAD", "SPEA2", "NSGAII", "PESA2", "PAES", "EpsMOEA"]

In [20]:
metamodel_algorithms = []
metamodel_algorithms_metamodels = [ 'GPR', 'RF', 'MLP'] 
metamodel_algorithms_strategies = ['NSGAII', "Random", "SMPSO"]

for metamodel in metamodel_algorithms_metamodels:
    for strategy in metamodel_algorithms_strategies:
        algo = metamodel + "_" + strategy
        print(algo)
        metamodel_algorithms += [algo]

GPR_NSGAII
GPR_Random
GPR_SMPSO
RF_NSGAII
RF_Random
RF_SMPSO
MLP_NSGAII
MLP_Random
MLP_SMPSO


In [21]:
algorithms.extend(metamodel_algorithms)

In [22]:
print(len(algorithms), algorithms)

19 ['MOEAD', 'SPEA2', 'NSGAII', 'OMOPSO', 'SMPSO', 'PESA2', 'PAES', 'EpsMOEA', 'GDE3', 'CMAES', 'GPR_NSGAII', 'GPR_Random', 'GPR_SMPSO', 'RF_NSGAII', 'RF_Random', 'RF_SMPSO', 'MLP_NSGAII', 'MLP_Random', 'MLP_SMPSO']


### Pareto Dominance Functionality
- `weakly_dominated`: determines if a certain solution is dominated or non-dominated.
- `get_non_dominated_ix`: creates a column describing whether a certain solution is dominated or not.
- `add_dominated_col`: adds a `isDominated` column to the dataframe

In [23]:
def weakly_dominates(v0, v1):
    return np.all(v0 <= v1) and np.any(v0 < v1)

In [24]:
def get_non_dominated(V):
    nsols, nobjs = V.shape
    
    dominated = np.zeros((nsols, 1))
    dominated_by = np.zeros((nsols, 1))
    
    for i in range(nsols):
        for j in range(nsols):
            if i != j:
                if weakly_dominates(V[j], V[i]):
                    dominated[i] = 1
                    dominated_by[i] = j 
                    break
                    
    return dominated, dominated_by
        
def add_isdominated_cols(d, cols=objs_cols):
    A = np.array(d[cols])
    B, C = get_non_dominated(A)
    d["isDominated"] = pd.DataFrame(B, columns=["isDominated"])
    d["dominatedBy"] = pd.DataFrame(C, columns=["dominatedBy"])
    print(d["isDominated"].value_counts())

### Collect the Pareto Front for each algorithm (APFs)

In [25]:
# Collect data for each algorithm
dfs = [get_algorithm_data(alg) for alg in algorithms]

Reading file: final/outputs/algorithms/all_MOEAD_solutions.csv
Reading file: final/outputs/algorithms/all_SPEA2_solutions.csv
Reading file: final/outputs/algorithms/all_NSGAII_solutions.csv
Reading file: final/outputs/algorithms/all_OMOPSO_solutions.csv
Reading file: final/outputs/algorithms/all_SMPSO_solutions.csv
Reading file: final/outputs/algorithms/all_PESA2_solutions.csv
Reading file: final/outputs/algorithms/all_PAES_solutions.csv
Reading file: final/outputs/algorithms/all_EpsMOEA_solutions.csv
Reading file: final/outputs/algorithms/all_GDE3_solutions.csv
Reading file: final/outputs/algorithms/all_CMAES_solutions.csv
Reading file: final/outputs/algorithms/all_GPR_NSGAII_solutions.csv
Reading file: final/outputs/algorithms/all_GPR_Random_solutions.csv
Reading file: final/outputs/algorithms/all_GPR_SMPSO_solutions.csv
Reading file: final/outputs/algorithms/all_RF_NSGAII_solutions.csv
Reading file: final/outputs/algorithms/all_RF_Random_solutions.csv
Reading file: final/outputs/alg

In [26]:
# Add isDominated column for each algorithm
for i in range(len(algorithms)):
    print("\t\tAlgorithm", algorithms[i], "\nRetrieved", len(dfs[i]), "solutions")
    dfs[i].drop(drop_cols, axis=1, inplace=True)
    add_isdominated_cols(dfs[i])
    dfs[i].sort_values(by=o1, inplace=True)

		Algorithm MOEAD 
Retrieved 762 solutions
1.0    719
0.0     43
Name: isDominated, dtype: int64
		Algorithm SPEA2 
Retrieved 699 solutions
1.0    670
0.0     29
Name: isDominated, dtype: int64
		Algorithm NSGAII 
Retrieved 713 solutions
1.0    693
0.0     20
Name: isDominated, dtype: int64
		Algorithm OMOPSO 
Retrieved 675 solutions
1.0    663
0.0     12
Name: isDominated, dtype: int64
		Algorithm SMPSO 
Retrieved 675 solutions
1.0    642
0.0     33
Name: isDominated, dtype: int64
		Algorithm PESA2 
Retrieved 713 solutions
1.0    677
0.0     36
Name: isDominated, dtype: int64
		Algorithm PAES 
Retrieved 675 solutions
1.0    600
0.0     75
Name: isDominated, dtype: int64
		Algorithm EpsMOEA 
Retrieved 677 solutions
1.0    635
0.0     42
Name: isDominated, dtype: int64
		Algorithm GDE3 
Retrieved 675 solutions
1.0    642
0.0     33
Name: isDominated, dtype: int64
		Algorithm CMAES 
Retrieved 1307 solutions
1.0    1270
0.0      37
Name: isDominated, dtype: int64
		Algorithm GPR_NSGAII 
R

In [27]:
# Get Non Dominated Solutions
APFs = [df[df['isDominated'] == 0] for df in dfs]

In [28]:
i = algorithms.index("GDE3")
APFs[i].head()

# moead: 1.192813	5.516787	-0.883130	20.340443	-1.479601	23.575989	1.038524 objs: 1.038524    55.978605
# smpso: 1.570796	31.415927	-1.570796	31.415927	1.454480	5.826618	0.576629	78.072770

,3,4,5,6,7,8,9,10,isDominated,dominatedBy
171,1.304655,5.906405,1.560995,31.415927,-1.217642,8.778142,0.695300,74.892021,0.0,0.0
180,1.304655,5.906405,1.560995,31.321804,-1.217642,12.627863,0.717500,72.880843,0.0,0.0
124,1.304655,3.102786,1.570796,31.415927,-0.570711,12.627863,0.724191,72.874952,0.0,0.0
169,1.304655,3.102786,1.570796,31.415927,-0.570711,14.179300,0.736950,72.598807,0.0,0.0
449,-1.570796,0.000000,1.380533,25.101777,1.041782,20.125805,0.775972,65.936823,0.0,0.0


### Collect True Pareto Front

In [188]:
all_sols_file = "all_solutions.csv"
r1_sols_file = "all_solutions_r1.csv"
r2_sols_file = "all_solutions_r2.csv"
r3_sols_file = "all_solutions_r3.csv"

In [189]:
def get_true_pf_data(file=all_sols_file, folder="final/outputs/"):
    filename = folder + file
    return pd.read_csv(filename, sep=',', header=None)

def process_pf(d):
    d.drop(drop_cols, axis=1, inplace=True)
    add_isdominated_cols(d)

In [190]:
TPF = get_true_pf_data()
process_pf(TPF)
print("True Pareto Front")
TPF.describe()
TPF.sort_values(by=o1, inplace=True)

1.0    13098
0.0       29
Name: isDominated, dtype: int64
True Pareto Front


In [175]:
TPF.sort_values(by=o1, inplace=True)
TPF.describe()

,3,4,5,6,7,8,9,10,isDominated,dominatedBy
count,13127.000000,13127.000000,13127.000000,13127.000000,13127.000000,13127.000000,13127.000000,13127.000000,13127.000000,13127.000000
mean,0.094451,11.305438,0.031115,12.269042,0.104918,11.424604,1.238065,36.455957,0.997791,135.200198
std,1.458253,8.594749,1.420494,8.676885,1.420706,7.888527,0.127045,15.030035,0.046952,620.651702
min,-3.141593,0.000000,-3.141593,0.000000,-3.141593,0.000000,0.576629,0.000000,0.000000,0.000000
25%,-1.076730,4.695171,-0.946134,5.205467,-0.856511,5.206773,1.164542,25.981562,1.000000,2.000000
50%,0.198772,9.227895,0.007949,10.857222,0.192907,10.340407,1.250986,38.197281,1.000000,16.000000
75%,1.209640,17.865079,1.057523,19.142639,1.047754,17.397197,1.324189,46.941621,1.000000,59.000000
max,3.141593,31.415927,3.141593,31.415927,3.141593,31.415927,1.857447,88.896425,1.000000,13080.000000


# Visual Processing

In [34]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.__version__

'3.7.1'

In [135]:
import plotly.io as pio

In [35]:
# plotly.tools.set_credentials_file(username='PastelBelem8', api_key='X8k1mloXRlB24rZm0qyq')
plotly.tools.set_credentials_file(username='KatarinaBelem', api_key='u5cJ4twW7lnZi128Bgjm')

In [209]:
colors = [
    # Verde
    'rgb(81, 142, 12)',
    # Vermelho
    'rgb(255, 0, 0)',
    # Roxo
    'rgb(152, 64, 188)',
    # Azul 
    'rgb(13, 188, 188)', 
    # Amarelo
    'rgb(255, 165, 21)',
    # Rosa
    'rgb(219, 31, 152)',
    # Cinzento claro
    'rgb(119, 130, 118)',
    # Azul escuro
    'rgb(0, 0, 255)',
    # Castanho
    'rgb(128, 64, 0)',
    
    # Metamodels
    # Bubblegum 1
    'rgb(128, 204, 204)',
    # Blanched almond 2
    'rgb(128, 238, 204)',
    # Goldenrod yellow 3
    'rgb(238, 128, 102)',
    # Mint Green 4
    'rgb(128, 255, 170)',
    # Electric Blue 5
    'rgb(102, 255, 255)',
    # Ultramarine blue 6
    'rgb(51, 119, 255)',
    # Pale cornflower blue 7
    'rgb(179, 191, 255)',
    # Lavender floral 8
    'rgb(170, 128, 255)',
    # Shocking pink 9
    'rgb(153,0,76)',
    # Preto
    'rgb(0, 100, 0)'
    ]

In [204]:
layout = go.Layout(
    # title = "Solutions of the minimization of two goals of an arc-shaped space frame",
    showlegend=True,
    # showlegend=False,
    autosize=False,
        # Definir tamanho do gráfico
    width=1000,
    height=600,
    legend=dict(orientation="v", 
                #xanchor="center", 
                #yanchor="bottom",
                #x= 0.5,
                #y=-0.5
               ),
    xaxis=dict(
        title="Maximum Displacement",
        range=[0.57, 1.42], 
        # autorange=True,
        showgrid=True,
        zeroline=False,
        showline=True,
        ticks='',
        showticklabels=True
    ),
    yaxis=dict(
        title="Euclidean Distance",
        range=[0, 82],
        # autorange=True,        
        showgrid=True,
        zeroline=False,
        showline=True,
        ticks='',
        showticklabels=True
    )
    )

In [213]:
def create_pf(algs=algorithms, colors=colors, layout=layout, approximatedPFs=APFs, combinedPf=TPF):
    # Create traces for each algorithm
    traces = []
    for i, a in enumerate(algorithms):
        trace = go.Scatter(
                    x = approximatedPFs[i][o1],
                    y = approximatedPFs[i][o2],
                    # mode = 'markers',
                    mode = 'lines+markers',
                    name = a,
                    marker=dict(
                        size=3.5,
                        color = colors[i],
                        opacity=1
                    ),
                    line=dict(
                        width=1,
                        color = colors[i]
                    ))
        traces += [trace]
     
    if combinedPf is not None:
        tpf_o1 = combinedPf[combinedPf['isDominated'] == 0][o1]
        tpf_o2 = combinedPf[combinedPf['isDominated'] == 0][o2]
    
        # Add True PF
        tpf_trace = go.Scatter(
            x = tpf_o1,
            y = tpf_o2,
            mode = 'markers',
            name = "All Algorithms",
            marker=dict(
                size=4.5,
                color = 'rgb(64, 64, 64)',
                opacity=1,
                line = dict(
                         color = 'rgb(64, 64, 64)',
                          width = 1
                )
                )
            )
        traces.append(tpf_trace)
    fig = go.Figure(data=traces, layout=layout)
    return fig, py.iplot(fig, filename='AllAlgorithmsPfs')

In [214]:
fig, graph = create_pf()
graph

In [158]:
pio.write_image(fig, 'all_pfs.svg', format='svg')

# All the points...

In [41]:
def create_dispersion_plot(algorithms=algorithms, colors=colors, layout=layout, data=dfs, combinedPf=TPF):
    # Create traces for each algorithm
    traces = []
    for i, a in enumerate(algorithms):
        datum = data[i]
        trace1 = go.Scatter(
                    x = datum[datum['isDominated'] == 0][o1],
                    y = datum[datum['isDominated'] == 0][o2],
                    #legendgroup = a,
                    name = a + " Non-dominated",
                    mode = 'lines+markers',
                    marker=dict(
                        size=5,
                        color = colors[i],
                        opacity=1
                    ),
                    line=dict(
                        color = colors[i]
                    ))
        trace2 = go.Scatter(
                    x = datum[datum['isDominated'] == 1][o1],
                    y = datum[datum['isDominated'] == 1][o2],
                    # legendgroup = a,
                    name = a + " dominated",
                    mode = 'markers',
                    marker=dict(
                        size=2.5,
                        color = colors[i],
                        opacity=0.8
                    ))

        traces += [trace1, trace2]
    if combinedPf is not None:
        tpf_o1 = combinedPf[combinedPf['isDominated'] == 0][o1]
        tpf_o2 = combinedPf[combinedPf['isDominated'] == 0][o2]

        tpf_trace = go.Scatter(
            x = tpf_o1,
            y = tpf_o2,
            mode = 'markers',
            name = "Combined Pareto Front",
            marker=dict(
                size=6,
                color = 'rgb(16, 46, 51)',
                opacity=1
                #line = dict(
               #     color = 'rgb(255, 255, 255)',
              #      width = 0.25
                 # )
                )
        )
        traces.append(tpf_trace)
    fig = go.Figure(data=traces, layout=layout)
    return py.iplot(fig, filename='ALL-TOGETHER-NOW')

In [42]:
create_dispersion_plot()

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~KatarinaBelem/0 or inside your plot.ly account where it is named 'ALL-TOGETHER-NOW'


In [172]:
def write_nondominated(data, file, folder=output_folder, header=False):
    data.to_csv(folder + file, header=header, index=False)

In [169]:
TPF_nd = TPF[TPF['isDominated']==0].copy()

In [170]:
TPF_nd.drop_duplicates(subset=objs_cols).describe()

,3,4,5,6,7,8,9,10,isDominated,dominatedBy
count,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.0,29.0
mean,0.608294,22.319318,0.019031,20.897644,0.031965,20.378585,1.062038,17.090279,0.0,0.0
std,1.076996,8.986277,1.136277,7.851800,1.179605,8.781610,0.240045,19.236103,0.0,0.0
min,-1.539420,5.157933,-1.570796,6.283185,-1.570796,5.253225,0.576629,0.000000,0.0,0.0
25%,0.777679,17.956169,-1.142830,16.632292,-1.195345,15.844965,0.921825,2.715903,0.0,0.0
50%,0.777679,18.102328,0.806111,17.969714,0.795472,17.397197,1.171189,7.671988,0.0,0.0
75%,1.484440,31.415927,0.806111,27.149803,0.795472,27.632602,1.261202,36.266240,0.0,0.0
max,1.570796,31.415927,1.570796,31.415927,1.570796,31.415927,1.308722,78.072770,0.0,0.0


In [173]:
write_nondominated(TPF_nd, "nondominated_o1_asc.csv")

In [191]:
def get_and_write(file, outfile):
    all_sols = get_true_pf_data(file)
    process_pf(all_sols)
    all_sols.sort_values(by=o1, inplace=True)
    all_sols_nd = all_sols[all_sols['isDominated']==0].copy()
    all_sols_nd.drop_duplicates(subset=objs_cols, inplace=True)
    write_nondominated(all_sols_nd, outfile)

In [192]:
get_and_write(r1_sols_file, "nondominated_r1_o1_asc.csv")

1.0    4332
0.0      14
Name: isDominated, dtype: int64


In [193]:
get_and_write(r2_sols_file, "nondominated_r2_o1_asc.csv")

1.0    4519
0.0      24
Name: isDominated, dtype: int64


In [194]:
get_and_write(r3_sols_file, "nondominated_r3_o1_asc.csv")

1.0    4219
0.0      19
Name: isDominated, dtype: int64


Contrastingly, when considering HV, if we compute the average of each run, PAES exhibits the largest volume,
hence strongly indicating that it is the best algorithm, with SMPSO, OMOPSO and MOEA/D
having the worst behaviors. However, when considering the combination of the three runs,
this situation is reversed and SMPSO and OMOPSO exhibit the best behaviors.

In [ ]:
df_extra_spea2 = df_extra_spea2.convert_objects(convert_numeric=True)
df_extra_spea2.info()

In [ ]:
# Additional run of SPEA2 for 1250 evaluations (50 generations of 25 individuals)
# df_extra_spea2_original = pd.read_csv("final/SPEA2_2000.csv", sep=',')
# Additional run of SPEA2 for 4000 evaluations (40 generations of 100 individuals)
df_extra_spea2_original = pd.read_csv("final/SPEA2_4000.csv", sep=',')

df_extra_spea2_original.drop(["Total time (s)", "Time o1", "Time o2"], axis=1, inplace=True)
df_extra_spea2 = df_extra_spea2_original.copy()

In [ ]:
df_extra_spea2[(df_extra_spea2["O1"] < 1) & (df_extra_spea2["O2"] < 60)].sort_values(by="O2")

In [ ]:
#df_extra_spea2.sort_values(by="O1", inplace=True)
add_isdominated_cols(df_extra_spea2, cols=["O1", "O2"])

In [ ]:
df_extra_spea2.sort_values(by="O1", inplace=True)
df_extra_spea2.head()

In [ ]:
traces3 = []
df_extra_spea2.sort_values(by="O1", inplace=True)
trace_extra_3 = go.Scatter(
                x = df_extra_spea2[df_extra_spea2['isDominated'] == 0]["O1"],
                y = df_extra_spea2[df_extra_spea2['isDominated'] == 0]["O2"],
                legendgroup = "Extra Spea2",
                name = "Extra SPEA2-4000 Non-dominated",
                mode = 'lines+markers',
                marker=dict(
                    size=5,
                    color = "rgb(0, 255, 0)",
                    opacity=1
                ),
                line=dict(
                    color = "rgb(0, 255, 0)"
                ))
trace_extra_4 = go.Scatter(
                x = df_extra_spea2[df_extra_spea2['isDominated'] == 1]["O1"],
                y = df_extra_spea2[df_extra_spea2['isDominated'] == 1]["O2"],
                legendgroup = "Extra Spea2",
                name = "Extra SPEA2-4000 dominated",
                mode = 'markers',
                marker=dict(
                    size=2.5,
                    color = "rgb(0, 255, 0)",
                    opacity=0.8
                ))

In [ ]:
traces3 = traces2 + [trace_extra_3, trace_extra_4]

In [ ]:
fig = go.Figure(data=traces3)
py.iplot(fig, filename='Extra Run')

In [ ]:
df_extra_spea2[(df_extra_spea2["O1"] <= 0.95) & (df_extra_spea2["O2"] <= 21)]

In [ ]:
df_extra_spea2.iloc[93:94]

In [ ]:
# Additional run of SPEA2 for 1250 evaluations (50 generations of 25 individuals)
# df_extra_spea2_original = pd.read_csv("final/SPEA2_2000.csv", sep=',')
# Additional run of SPEA2 for 4000 evaluations (40 generations of 100 individuals)
df_extra_smpso_original = pd.read_csv("final/outputs/all_SMPSO_2_solutions.csv", sep=',', header=None)
df_extra_smpso_original.head()

In [ ]:
#df_extra_spea2.sort_values(by="O1", inplace=True)
add_isdominated_cols(df_extra_smpso_original)

In [ ]:
traces4 = []
df_extra_smpso_original.sort_values(by=9, inplace=True)
trace_extra_5 = go.Scatter(
                x = df_extra_smpso_original[df_extra_smpso_original['isDominated'] == 0][o1],
                y = df_extra_smpso_original[df_extra_smpso_original['isDominated'] == 0][o2],
                legendgroup = "Extra SMPSO",
                name = "Extra SMPSO Non-dominated",
                mode = 'lines+markers',
                marker=dict(
                    size=5,
                    color = "rgb(255, 0, 0)",
                    opacity=1
                ),
                line=dict(
                    color = "rgb(255, 0, 0)"
                ))
trace_extra_6 = go.Scatter(
                x = df_extra_smpso_original[df_extra_smpso_original['isDominated'] == 1][o1],
                y = df_extra_smpso_original[df_extra_smpso_original['isDominated'] == 1][o2],
                legendgroup = "Extra SMPSO",
                name = "Extra SMPSO dominated",
                mode = 'markers',
                marker=dict(
                    size=2.5,
                    color = "rgb(255, 0, 0)",
                    opacity=0.8
                ))

In [ ]:
traces4 = traces3 + [trace_extra_5, trace_extra_6]

In [ ]:
fig = go.Figure(data=traces4)
py.iplot(fig, filename='Extra Run')

###  Check Min and Max Values per iteration...

In [ ]:
pop_size = 25
nrows=df_extra_spea2.shape[0]
nrows

In [ ]:
n_iterations = nrows // pop_size
n_iterations

In [ ]:
df_extra_spea2_original.iloc[0:2] # Syntax Example

In [ ]:
df_iterations = []
for i in range(n_iterations + 1):
    i_in, i_end = i * pop_size, i*pop_size+1
    df_iterations += [df_extra_spea2_original.loc[i_in:i_end].copy()]

In [ ]:
len(df_iterations)

In [ ]:
df_iterations

In [ ]:
iterations_min = pd.DataFrame(columns=df_iterations[1].columns)
for iteration in df_iterations:
    iterations_min.append(iteration.min(), ignore_index=True)
iterations_min

In [ ]:
df_iterations

# Indicators Processing

In [225]:
indicators_df = pd.read_csv("final/outputs/indicators-HV-Max-20190413192210.csv", sep=',')
indicators_df[(indicators_df["Algorithm"] == "MOEAD")|(indicators_df["Algorithm"] == "SMPSO")]

,Algorithm,Run,HV,ONVG,SPACING,SPREAD,MAXIMUM SPREAD,ONVGR,Error Ratio,Max Pareto Front Error,GD,IGD
3,SMPSO,1,0.659228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SMPSO,2,0.750527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,SMPSO,3,0.765932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,MOEAD,1,0.527267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,MOEAD,2,0.545952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MOEAD,3,0.512304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [226]:
indicators_df.describe()

,Run,HV,ONVG,SPACING,SPREAD,MAXIMUM SPREAD,ONVGR,Error Ratio,Max Pareto Front Error,GD,IGD
count,57.000000,57.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,2.000000,0.560394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.823754,0.074107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,0.399379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.000000,0.522168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,2.000000,0.542021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,3.000000,0.596769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,3.000000,0.765932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [227]:
# Create new DF with mean and std deviation
indicators_mean = indicators_df.groupby(["Algorithm"]).mean()
indicators_mean.sort_values(by="IGD", inplace=False).to_csv("final/outputs/indicators-HV-Max-20190413192210_MEAN.csv")# [["IGD", "HV"]]

In [228]:
indicators_std = indicators_df.groupby(["Algorithm"]).std()
indicators_std.to_csv("final/outputs/indicators-HV-Max-20190413192210_STD.csv")

In [ ]:
# HV --> The larger, the better
# ONVG --> The larger, the better (number of non-dominated solutions per run)

## Combined PFs indicators

In [ ]:
combined_indicators_df = pd.read_csv("final/outputs/indicators_3_runs_combined.csv", sep=',')
combined_indicators_df.sort_values(by="IGD", inplace=False)[["IGD", "HV", "Algorithm"]]

In [ ]:
combined_indicators_df

# Test Click Events
I believe this currently works on online settings, but I have tried in the offline scenario with no success.

In [ ]:
tpf_o1 = TPF[TPF['isDominated'] == 0][o1]
tpf_o2 = TPF[TPF['isDominated'] == 0][o2]
    
tpf_vars = TPF[TPF['isDominated'] == 0][vars_cols]


In [ ]:
# Add True PF
tpf_trace = go.Scatter(
    x = tpf_o1,
    y = tpf_o2,
    mode = 'lines+markers',
    customdata = tpf_vars,
    name = "Combined Pareto Front",
    marker=dict(
        size=6,
        color = 'rgb(13, 188, 188)', #'rgb(16, 46, 51)',
        opacity=1
        #line = dict(
        #     color = 'rgb(255, 255, 255)',
        #      width = 0.25
        # )
        )
)

In [ ]:
f = go.FigureWidget([tpf_trace])

In [ ]:
scatter = f.data[0]

In [ ]:
f.layout.hovermode = 'closest'

In [ ]:
# create our callback function
def update_point(trace, points, selector):
    print(points.point_inds)
    
    clickedPoints = [print(trace['customdata'][points.point_inds][:])
    
    #for i in points.point_inds:

scatter.on_click(update_point)

In [ ]:
f